In [3]:
import math
import numpy as np
import os
import pandas as pd
import pickle
from PIL import Image
from sklearn.metrics import confusion_matrix, matthews_corrcoef
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2Model, pipeline, AutoModelForCausalLM, \
AutoTokenizer, BitsAndBytesConfig, LlamaTokenizer, LlamaModel, AutoModelForTextEncoding
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer
import random
seed_value = 42
torch.manual_seed(seed_value)
np.random.seed(seed_value)

random.seed(seed_value)



In [22]:
import torch.nn as nn
import torch.optim as optim

class GPT_CNN2D(nn.Module):
    def __init__(self, embedding_model):
        super(GPT_CNN2D, self).__init__()
        self.em_model = embedding_model
        
            
        if embedding_model == "custom":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=8, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()            
            # self.fc1 = nn.Linear(96, 2)
            self.fc1 = nn.Linear(384, 2)
            # self.fc2 = nn.Linear(128, 2)
            self.relu = nn.ReLU()

    def forward(self, x):
        
        if self.em_model == "custom":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT_CNN2D(embedding_model = "custom").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

In [24]:
import torch
from transformers import BertModel
# Create an instance of the model
# Map the location to the first CUDA device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pretrained_weights = torch.load('/home/csgrad/kaushik3/LLM/Code/Embedding_Fusion/Code/gpt_BERT_FLAN.pth', map_location=device)
model.load_state_dict(pretrained_weights)


# Now the model is loaded with the pretrained weights


<All keys matched successfully>

In [9]:
print(device)

cuda:0


In [10]:
from datasets import load_dataset
dataset = load_dataset("yaful/DeepfakeTextDetect")

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [11]:
human_train = [data for data in dataset['train'] if data['src'] == 'squad_human']
human_test = [data for data in dataset['test'] if data['src'] == 'squad_human']
human_val = [data for data in dataset['validation'] if data['src'] == 'squad_human']

llm_train = [data for data in dataset['train'] if 'gpt' in data['src'] and 'squad' in data['src']]
llm_test = [data for data in dataset['test'] if 'gpt' in data['src'] and 'squad' in data['src']]
llm_val = [data for data in dataset['validation'] if 'gpt' in data['src'] and 'squad' in data['src']]

import random

min_train_length = min(len(human_train), len(llm_train))
min_test_length = min(len(human_test), len(llm_test))
min_val_length = min(len(human_val), len(llm_val))

squad_human_train = random.sample(human_train, min_train_length)
squad_human_test = random.sample(human_test, min_test_length)
squad_human_val = random.sample(human_val, min_val_length)

squad_gpt_train = random.sample(llm_train, min_train_length)
squad_gpt_test = random.sample(llm_train, min_test_length)
squad_gpt_val = random.sample(llm_val, min_val_length)

print(len(human_train), len(llm_train))
print(len(squad_human_train), len(squad_gpt_train))
print(len(squad_human_test), len(squad_gpt_test))
print(len(squad_human_val), len(squad_gpt_val))

15820 1830
1830 1830
228 228
220 220


In [12]:
import pandas as pd

Human = pd.DataFrame()
squad_human = squad_human_train + squad_human_val + squad_human_test
Human['text'] = [data['text'] for data in squad_human]

AI = pd.DataFrame()
squad_gpt = squad_gpt_train + squad_gpt_val + squad_gpt_test
AI['text'] = [data['text'] for data in squad_gpt]

In [13]:
Human = Human['text'].tolist()
AI = AI['text'].tolist()

In [40]:
Human[0]

'Gratitude for all these resources and the determination to develop oneself would be more productive than criticism and blame because the resources are readily available and because, if you blame others, there is no need for you to do something different tomorrow or for you to change and improve. Where there is a will, there is a way. People in developed countries have the will and the way to do many things that they want to do. They sometimes need more determination and will to improve and to educate themselves with the resources that are abundantly available. They occasionally need more gratitude for the resources they have, including their teachers and their textbooks. The entire internet is also available to supplement these teachers and textbooks.'

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to('cuda')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
input_ids = tokenizer.encode(Human[0], return_tensors='pt',padding=True, truncation=True, max_length=512).to(device)


In [63]:
print(input_ids.shape)

torch.Size([1, 137])


In [14]:
model_gpt = GPT2Model.from_pretrained('gpt2') 
model_gpt = model_gpt.to('cuda')
tokenizer_gpt = GPT2Tokenizer.from_pretrained('gpt2')
model_name = "roberta-large"
model_bert = RobertaModel.from_pretrained(model_name)
model_bert = model_bert.to('cuda')

# Load the tokenizer
tokenizer_bert = RobertaTokenizer.from_pretrained(model_name)

tokenizer_flan = AutoTokenizer.from_pretrained("/data/kaushik3/models--pszemraj--flan-ul2-text-encoder/snapshots/1ffcec2cc23aa34ea6edb96f86c12f179ee9e87f")
model_flan = AutoModelForTextEncoding.from_pretrained("/data/kaushik3/models--pszemraj--flan-ul2-text-encoder/snapshots/1ffcec2cc23aa34ea6edb96f86c12f179ee9e87f")
model_flan = model_flan.to('cuda')
    


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to('cuda')
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) 

for val in range(len(Human)):
    
    if val%100==0:
        print(val)
    
    input_ids = tokenizer.encode(Human[val], return_tensors='pt',padding=True, truncation=True, max_length=512).to(device) 
    #inputs = {name: tensor.to('cuda') for name, tensor in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(input_ids)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        #last_hidden_states = outputs.last_hidden_state
        embeddings = outputs.last_hidden_state.mean(dim = 1).squeeze().to('cpu')

In [15]:
human_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/human_embeddings.npy', allow_pickle=True)
human_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/GPT/human_embeddings.npy', allow_pickle=True)
human_flan = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/human_embeddings.npy', allow_pickle=True)


ai_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/gpt_embeddings.npy', allow_pickle=True)
ai_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/GPT/gpt_embeddings.npy', allow_pickle=True)
ai_flan = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/gpt_embeddings.npy', allow_pickle=True)


bert_min_length = min(len(human_bert), len(ai_bert))
human_bert = human_bert[:bert_min_length]
ai_bert = ai_bert[:bert_min_length]

gpt_min_length = min(len(human_gpt), len(ai_gpt))
human_gpt = human_gpt[:bert_min_length]
ai_gpt = ai_gpt[:bert_min_length]

flan_min_length = min(len(human_flan), len(ai_flan))
human_flan = human_flan[:bert_min_length]
ai_flan = ai_flan[:bert_min_length]


print(len(human_bert), len(human_gpt), len(human_flan), len(ai_bert), len(ai_gpt), len(ai_flan))

human_bert = list(human_bert)
ai_bert = list(ai_bert)
human_gpt = list(human_gpt)
ai_gpt = list(ai_gpt)
human_flan = list(human_flan)
ai_flan = list(ai_flan)

human_bert = torch.tensor(human_bert)
ai_bert = torch.tensor(ai_bert)
human_gpt = torch.tensor(human_gpt).reshape([bert_min_length, 1, 768])
ai_gpt = torch.tensor(ai_gpt).reshape([bert_min_length, 1, 768])
human_flan = torch.tensor(human_flan).reshape([bert_min_length, 1, 4096])
ai_flan = torch.tensor(ai_flan).reshape([bert_min_length, 1, 4096])


human_bert = human_bert.view(bert_min_length, 16,64)
human_gpt = human_gpt.view(bert_min_length, 12,64)
human_flan = human_flan.view(bert_min_length, 64,64)

ai_bert = ai_bert.view(bert_min_length, 16,64)
ai_gpt = ai_gpt.view(bert_min_length, 12,64)
ai_flan = ai_flan.view(bert_min_length, 64,64)

human = torch.cat([human_bert, human_gpt, human_flan], dim=1)
ai = torch.cat([ai_bert, ai_gpt, ai_flan], dim=1)
embeds = torch.cat((human, ai), dim=0)
zeros_tensor = torch.zeros((bert_min_length))
ones_tensor = torch.ones((bert_min_length))

# Concatenate the tensors along the first dimension
labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

# Display the result tensor
print(labels)

2278 2278 2278 2278 2278 2278


/tmp/ipykernel_1033269/3304321486.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  human_bert = torch.tensor(human_bert)


tensor([0., 0., 0.,  ..., 1., 1., 1.])


In [16]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 32

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
import torch

import torch.nn as nn
import torch.optim as optim


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GPT_CNN2D(embedding_model = "fusion3").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
max_val_accuracy = 0
# Step 5: Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    if max_val_accuracy<val_accuracy:
        max_TP = TP
        max_FP = FP
        max_TN = TN
        max_FN = FN
    max_val_accuracy = max(max_val_accuracy, val_accuracy)
    
    
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x480 and 384x2)